### Hybrid Pipeline 雙階段混和
> 非監督篩式清洗 + 監督式分類

> autoencoder(AE) + catboost

#### 載入外部程式庫

anaconda 需額外載入
> tensorflow

> catboost

In [1]:
import pandas as pd
from pandas.testing import assert_frame_equal # panda 檢查
# model
from src.data_processing import DataProcessor
# Evaluater
import sys
import os


#### 準備資料集

##### Load csv


In [2]:
entities_path = {
     "T1"   :   "../DataSet/preprocessing_T1_basic.csv"
    ,"T12"  :   "../DataSet/preprocessing_T1_2.csv"
    ,"T123" :   "../DataSet/preprocessing_T1_2_3.csv"
}
entities = {}

# Load preprocessing and cut 
for index, path in entities_path.items():
    entities[index] = pd.read_csv(path)

print("原始資料：")
for v in entities.values():
    print(f"{v.head()}\n\n")

原始資料：
                                                acct  txn_count  first_txn_ts  \
0  00000577cfcd0bde8ee693021419ef13a1f7f933ec8626...          1       1953000   
1  00000eec52ea49377de91bc7b54eb3192943e6c20e0a51...          1       4489500   
2  000015150c92e2a41c4715a088df78d77a7d4f3017aadc...          1       5823000   
3  00002846e6b430580825e2b10fe3ff1e3ddb93f42c608d...          1       1100100   
4  00002b3d8f9c7b91c407a5725849deb521fcf1dd5eea1f...          1        777300   

   last_txn_ts  std_txn_ts  acct_type_x  cross_type    uni_amt  acct_type_y  \
0      1953000         0.0            2           1    47500.0            2   
1      4489500         0.0            2           1     6150.0            2   
2      5823000         0.0            2           1  1150000.0            2   
3      1100100         0.0            2           1     8550.0            2   
4       777300         0.0            2           1     1450.0            2   

   num_unique_dest_accts  num_un

##### Cut acct ID

In [3]:
# Cut preprocessing
entities_id = pd.DataFrame()

for index, entity in entities.items():
    id = entity.iloc[:,[0]]
    entities[index] = entity.iloc[:,1:]

    # 判斷entity id 是否相同
    if entities_id.empty:
        entities_id = id
    else :
        try:
            assert_frame_equal(entities_id, id)
        except AssertionError as e:
            print(f"Data ID error : from {index}")
            print(e)

print("資料ID :")
entities_id.head()

資料ID :


,acct
0,00000577cfcd0bde8ee693021419ef13a1f7f933ec8626...
1,00000eec52ea49377de91bc7b54eb3192943e6c20e0a51...
2,000015150c92e2a41c4715a088df78d77a7d4f3017aadc...
3,00002846e6b430580825e2b10fe3ff1e3ddb93f42c608d...
4,00002b3d8f9c7b91c407a5725849deb521fcf1dd5eea1f...


#### 建立 Evaluater評分 (結果產出)

In [4]:
def get_evaluater_score(model,X_train, X_test, y_train, y_test):
    # 假設您的環境設定
    sys.path.append(os.path.dirname(os.getcwd()))
    from Util import Evaluater

    Evaluater.evaluate_model(model, (X_train, X_test, y_train, y_test))

#### 實作 : 定義運行資料集(dict)

In [7]:
# entities 標籤 (dict)
entities_index = []
for index in entities:
    entities_index.append(index)

print(entities_index)

['T1', 'T12', 'T123']


In [8]:
#==== 實作引用資料參考上面 輸出 index ====#

#       這裡手動調整跑訓練的資料集(參考上方輸出)
#                       vv
train_entities_index = ["T1","T12","T123"]
#                       ^^
#       這裡手動調整跑訓練的資料集(參考上方輸出)

#       這裡手動調整跑訓練的次數
#                  vv
model_train_turn = 50
#                  ^^
#       這裡手動調整跑訓練的次數

In [9]:
# 定義輸出資料集
best_models = {}

# 定義訓練/測試資料庫
cutted_data = {}

#### 實作 : 資料分析 ( AE + CatBoost )

In [10]:
from src.model import AE_CatBoost_Model
sys.path.append(os.path.dirname(os.getcwd()))
from Util.PrepareData import prepare_data_pure,prepare_data_cutting,prepare_data_smote


In [ ]:
for index in train_entities_index:
    # 切資料
    print(f"\nindex = {index}")
    X_train, X_test, y_train, y_test =  prepare_data_cutting(entity)
    cutted_data[index] = X_train, X_test, y_train, y_test
    
    # show label num
    actual_cols = X_test.shape[1] if len(X_test.shape) > 1 else 1
    print(f"測試資料 (X_test) 的特徵數量: {actual_cols}")

index = T1
訓練集大小: (129451, 12), 測試集大小: (54275, 12)
訓練集正類比例: 2.72%, 測試集正類比例: 0.55%
測試資料 (X_test) 的特徵數量: 12
index = T12
訓練集大小: (129451, 17), 測試集大小: (54275, 17)
訓練集正類比例: 2.72%, 測試集正類比例: 0.55%
測試資料 (X_test) 的特徵數量: 17
index = T123
訓練集大小: (129451, 33), 測試集大小: (54275, 33)
訓練集正類比例: 2.72%, 測試集正類比例: 0.55%
測試資料 (X_test) 的特徵數量: 33


In [ ]:
for index in train_entities_index:
    print(f"\nindex = {index}")
    # init model & 資料暫存庫
    model = AE_CatBoost_Model()
    best_models[index] = None
    entity = entities[index]

    # 切分訓練與測試資料(第一次切分 : 統一切分)
    X_train, X_test, y_train, y_test =  cutted_data[index]

    # 訓練模型
    model.fit(X_train, y_train) 
    # model.fit(X_train, y_train, train_turn=20,tune_params=True)
        
    # 儲存最佳模型
    best_models[index] = model



index = T1
測試資料 (X_test) 的特徵數量: 12
測試資料 (X_train) 的特徵數量: 12
步驟 1: 訓練 AutoEncoder 並篩選樣本...
4046/4046 ━━━━━━━━━━━━━━━━━━━━ 2s 515us/step
篩選出 103560 筆可靠正常樣本，剔除 25891 筆潛在雜訊。

步驟 2: 建立混合資料集並訓練 CatBoost...

CatBoost 訓練完成
模型預期的特徵數量 (訓練時): 33

index = T12
測試資料 (X_test) 的特徵數量: 17
測試資料 (X_train) 的特徵數量: 17
步驟 1: 訓練 AutoEncoder 並篩選樣本...
4046/4046 ━━━━━━━━━━━━━━━━━━━━ 2s 574us/step
篩選出 103560 筆可靠正常樣本，剔除 25891 筆潛在雜訊。

步驟 2: 建立混合資料集並訓練 CatBoost...

CatBoost 訓練完成
模型預期的特徵數量 (訓練時): 33

index = T123
測試資料 (X_test) 的特徵數量: 33
測試資料 (X_train) 的特徵數量: 33
步驟 1: 訓練 AutoEncoder 並篩選樣本...
4046/4046 ━━━━━━━━━━━━━━━━━━━━ 2s 604us/step
篩選出 103560 筆可靠正常樣本，剔除 25891 筆潛在雜訊。

步驟 2: 建立混合資料集並訓練 CatBoost...

CatBoost 訓練完成
模型預期的特徵數量 (訓練時): 33


In [27]:
for index in train_entities_index:
    print(f"index = {index}")
    # 重切資料
    X_train, X_test, y_train, y_test =  cutted_data[index]
    
    # 1. 取得模型物件
    current_model = best_models[index].cat_model

    # 2. 檢查模型訓練時用了幾個特徵
    # 優先嘗試 n_features_in_，如果沒有則嘗試計算 feature_names_
    if hasattr(current_model, 'n_features_in_'):
        expected_cols = current_model.n_features_in_
    elif hasattr(current_model, 'feature_names_'):
        expected_cols = len(current_model.feature_names_)
    else:
        expected_cols = "未知 (無法讀取屬性)"

    print(f"模型預期的特徵數量 (訓練時): {expected_cols}")

    # 3. 檢查現在傳入的資料有幾個特徵
    actual_cols = X_test.shape[1] if len(X_test.shape) > 1 else 1
    print(f"測試資料 (X_test) 的特徵數量: {actual_cols}")

    # 4. 自動判斷
    if isinstance(expected_cols, int):
        if expected_cols != actual_cols:
            diff = expected_cols - actual_cols
            if diff > 0:
                print(f"\n[錯誤原因] 測試資料少給了 {diff} 個欄位！")
                print("請檢查：是否在訓練時有加入 'AE 重建誤差' 或其他特徵，但測試時忘記加了？")
            else:
                print(f"\n[錯誤原因] 測試資料多出了 {abs(diff)} 個欄位！")
                print("請檢查：測試資料是否忘記刪除 ID 欄位或 Label 欄位？")
        else:
            print("\n[狀態] 欄位數量一致。如果還是報錯，請確認是否為 DataFrame vs Array 的格式問題 (請使用 .values)。")

    # evaluater
    print ("\n"+"-"*50)
    print ("evaluater 測試：")
    print ("-"*50)
    get_evaluater_score(best_models[index],X_train, X_test, y_train, y_test)
        
    # 找回 label = 1 的準確度
    """
    - entity_label1 : 全部為 label 1 的資料集
    - X_label1, y_label1 : 從 entity_label1 切分出來的訓練資料 (All label 1)
    - X_test_label1, y__test_label1 : 從 X_test 切分出來的測試資料 (All label 1)
    """
    # All label 1
    entity_label1 = entity.copy()
    label,entity_label1_cut = DataProcessor.cut_label(entity_label1) # label -> enitiy的 label ; entity_label1_cut -> entity 移除 label 列 
    X_label1 = DataProcessor.split_diff_label(entity_label1_cut, label, positive_label=True) # label1 -> entity 移除 label 列並取 label = 1 (alert)
    y_label1 = pd.Series([1]*X_label1.shape[0]) # 全部為 1
    # no train label 1
    X_test_label1 = DataProcessor.split_diff_label(X_test, y_test, positive_label=True) # 從 X_test 切分出來的測試資料 (All label 1) 
    y_test_label1 = pd.Series([1]*X_test_label1.shape[0])

    # 產生分數
    print ("\n"+"-"*50)
    print ("Label = 1 (alert) 的準確度：")
    print ("-"*50)
    get_evaluater_score(best_models[index],X_train, X_label1, y_train, y_label1)

index = T1
模型預期的特徵數量 (訓練時): 33
測試資料 (X_test) 的特徵數量: 12

[錯誤原因] 測試資料少給了 21 個欄位！
請檢查：是否在訓練時有加入 'AE 重建誤差' 或其他特徵，但測試時忘記加了？

--------------------------------------------------
evaluater 測試：
--------------------------------------------------


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 12 is present in model but not in pool.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import precision_recall_curve
import numpy as np

for index in train_entities_index:
        
    # 重切資料
    X_train, X_test, y_train, y_test =  cutted_data[index]

    # show label num
    actual_cols = X_test.shape[1] if len(X_test.shape) > 1 else 1
    print(f"測試資料 (X_test) 的特徵數量: {actual_cols}")

    # 1. 取得預測機率
    # 務必使用測試集 (Test Set)
    y_probs = model.predict_proba(X_test)[:, 1]

    # 2. 建立畫布
    fig, ax = plt.subplots(figsize=(10, 7))

    # 3. 使用方法一畫出基礎曲線 (Blue Line)
    display = PrecisionRecallDisplay.from_predictions(
        y_test, 
        y_probs, 
        name="AE + CatBoost",
        plot_chance_level=True,
        ax=ax
    )

    # =======================================================
    # 新增功能：計算並標示「最佳 F1-Score」的點
    # =======================================================

    # 4. 手動計算 Precision, Recall 和 Thresholds 來找最佳解
    precision, recall, thresholds = precision_recall_curve(y_test, y_probs)

    # 5. 計算每個點的 F1-Score
    # F1 = 2 * (P * R) / (P + R)
    # 加上 1e-8 是為了避免分母為 0 報錯
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)

    # 6. 找出 F1 最高點的索引 (Index)
    best_idx = np.argmax(f1_scores)

    best_p = precision[best_idx]
    best_r = recall[best_idx]
    best_f1 = f1_scores[best_idx]

    # 注意：thresholds 陣列的長度比 precision/recall 少 1
    # 如果 best_idx 是最後一個，代表閾值非常高
    best_thresh = thresholds[best_idx] if best_idx < len(thresholds) else 1.0

    # 7. 在原本的圖上疊加一個紅點
    ax.scatter(best_r, best_p, s=150, c='red', edgecolors='black', zorder=10, 
            label=f'Best F1 (Th={best_thresh:.3f})')

    # 8. 加上文字註解
    text_label = f'Best F1={best_f1:.2f}\nRecall={best_r:.2f}\nPrecision={best_p:.2f}\nThreshold={best_thresh:.3f}'

    ax.annotate(text_label, 
            xy=(best_r, best_p), 
            xytext=(30, 40), # 文字稍微偏右上
            textcoords='offset points',
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.9),
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))

    # =======================================================

    ax.set_title("Precision-Recall Curve with Best F1 Point")
    ax.grid(linestyle="--", alpha=0.6)
    plt.legend(loc="lower left") # 圖例放左下角避免擋住線
    plt.show()